In [1]:
#Project 3 - Notebook 3 (h2o AutoML)

In [2]:
#Dependencies
import numpy as np
import os
import psycopg2
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
import h2o

In [4]:
#Cloud DB Config
SQL_USERNAME = "postgres" #your personal username goes here
SQL_PASSWORD = "airlineproject" #your personal password goes here
DB_NAME = "airline_db" #The default name of the DB
DB_HOST = "airline-db.c0dqcvhs1y58.us-east-2.rds.amazonaws.com"
PORT = 5432
database_path = f"postgresql://{SQL_USERNAME}:{SQL_PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}"

In [5]:
from sqlalchemy import create_engine
engine = create_engine(database_path)

In [ ]:
#main_df = pd.read_sql_query('select "Mkt_Ccode", "Arr_Delay_Time_Actual" from main_airline;', con=engine)
info_df = pd.read_sql_query('select * from main_airline FETCH FIRST 5 ROWS ONLY;', con=engine)

In [ ]:
test_df = pd.read_sql_query('select * from main_airline order by random() limit 10000;', con=engine)

In [ ]:
test_df.info()

In [ ]:
test = test_df.drop(columns = ["index", "Gate_Dep_Time_Actual", "Arr_Delay_Time_Actual", "Dep_Delay_Time_Actual", "Gate_Arr_Time_Actual"])

In [ ]:
test.head()

In [ ]:
train_df = pd.read_sql_query('select * from main_airline order by random() limit 100000;', con=engine)

In [ ]:
train = train_df.drop(columns = ["index", "Gate_Dep_Time_Actual", "Arr_Delay_Time_Actual", "Dep_Delay_Time_Actual", "Gate_Arr_Time_Actual"])


In [ ]:
test.to_csv('test.csv', index=False)
train.to_csv('train.csv', index=False)

In [6]:
import h2o
from h2o.automl import H2OAutoML

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\dlken\AppData\Local\Temp\tmpg58nu3r1
  JVM stdout: C:\Users\dlken\AppData\Local\Temp\tmpg58nu3r1\h2o_dlken_started_from_python.out
  JVM stderr: C:\Users\dlken\AppData\Local\Temp\tmpg58nu3r1\h2o_dlken_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,08 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_dlken_gemh6f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.754 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

In [ ]:
x = train.columns
y = "Arr_Delay_Time_Actual_Flag"
x.remove(y)

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 100, seed=1)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
aml.leader

2nd Try with Corey's data setup

In [9]:
test2_df = pd.read_sql_query('select "Mkt_Ccode", "Dep_Airport", "Arr_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block", "Arr_Delay_Time_Actual_Flag" from main_airline order by random() limit 100000;', con=engine)

In [8]:
train2_df = pd.read_sql_query('select "Mkt_Ccode", "Dep_Airport", "Arr_Airport", "Day_Week", "month", "Sched_Dep_Time_OAG_Block", "Arr_Delay_Time_Actual_Flag" from main_airline order by random() limit 750000;', con=engine)

In [ ]:
test2_df.head()

In [10]:
test2_df.to_csv('test2.csv', index=False)
train2_df.to_csv('train2.csv', index=False)

In [11]:
train = h2o.import_file("train2.csv")
test = h2o.import_file("test2.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
x = train.columns
y = "Arr_Delay_Time_Actual_Flag"
x.remove(y)

In [13]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [20]:
aml = H2OAutoML(max_runtime_secs = 600, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
19:00:01.822: AutoML: XGBoost is not available; skipping it.

██████████████████████
19:03:48.979: GBM_5_AutoML_20210824_190001 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20210824_190001_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (104.7 KB per tree x 10000 > 408.7 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!


██████████████████████████████████| 100%


In [22]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210824_190001,0.622943,0.62307,0.457422,0.43946,0.465201,0.216412
StackedEnsemble_BestOfFamily_AutoML_20210824_190001,0.621857,0.623433,0.456412,0.439342,0.465375,0.216574
GBM_grid__1_AutoML_20210824_190001_model_2,0.621491,0.624381,0.455847,0.441809,0.46583,0.216998
GBM_2_AutoML_20210824_190001,0.617246,0.62844,0.450355,0.441649,0.467772,0.21881
GBM_1_AutoML_20210824_190001,0.617025,0.627193,0.450889,0.448919,0.467175,0.218253
GBM_4_AutoML_20210824_190001,0.616138,0.631039,0.448093,0.446795,0.46902,0.219979
GBM_3_AutoML_20210824_190001,0.616132,0.629003,0.448637,0.446644,0.468042,0.219063
GBM_grid__1_AutoML_20210824_190001_model_1,0.615076,0.626089,0.448634,0.446348,0.466664,0.217775
GLM_1_AutoML_20210824_190001,0.605769,0.629125,0.438344,0.458634,0.468072,0.219091
DeepLearning_grid__1_AutoML_20210824_190001_model_1,0.592938,0.632027,0.430665,0.47467,0.469458,0.220391


In [23]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210824_190001

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.2126372847160814
RMSE: 0.4611261050039147
LogLoss: 0.6141087537696212
Null degrees of freedom: 9931
Residual degrees of freedom: 9921
Null deviance: 12843.667011010704
Residual deviance: 12198.656284879755
AIC: 12220.656284879755
AUC: 0.6539293919320116
AUCPR: 0.49091107123739525
Gini: 0.3078587838640232

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2983879969974675: 


,,0,1,Error,Rate
0,0,2746.0,3724.0,0.5756,(3724.0/6470.0)
1,1,752.0,2710.0,0.2172,(752.0/3462.0)
2,Total,3498.0,6434.0,0.4507,(4476.0/9932.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.298388,0.547696,263.0
1,max f2,0.218834,0.731887,344.0
2,max f0point5,0.385359,0.486171,171.0
3,max accuracy,0.478961,0.668848,87.0
4,max precision,0.711839,1.000000,0.0
5,max recall,0.141054,1.000000,389.0
6,max specificity,0.711839,1.000000,0.0
7,max absolute_mcc,0.327036,0.210935,232.0
8,max min_per_class_accuracy,0.344995,0.606801,213.0
9,max mean_per_class_accuracy,0.327036,0.610433,232.0



Gains/Lift Table: Avg response rate: 34.86 %, avg score: 34.53 %


...

,,0,1,Error,Rate
0,0,133356.0,357637.0,0.7284,(357637.0/490993.0)
1,1,38987.0,220020.0,0.1505,(38987.0/259007.0)
2,Total,172343.0,577657.0,0.5288,(396624.0/750000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.268286,0.525946,291.0
1,max f2,0.044048,0.725092,399.0
2,max f0point5,0.370829,0.456807,189.0
3,max accuracy,0.491272,0.662328,90.0
4,max precision,0.794761,0.818182,2.0
5,max recall,0.044048,1.000000,399.0
6,max specificity,0.925981,0.999998,0.0
7,max absolute_mcc,0.354812,0.169853,204.0
8,max min_per_class_accuracy,0.344170,0.587834,215.0
9,max mean_per_class_accuracy,0.351495,0.588546,208.0



Gains/Lift Table: Avg response rate: 34.53 %, avg score: 34.53 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.588577,1.817711,1.817711,0.627733,0.625179,0.627733,0.625179,0.018177,0.018177,81.771149,81.771149,0.012491
1,2,0.02,0.560575,1.687599,1.752655,0.582800,0.573289,0.605267,0.599234,0.016876,0.035053,68.759918,75.265533,0.022994
2,3,0.03,0.542571,1.621578,1.708963,0.560000,0.551078,0.590178,0.583182,0.016216,0.051269,62.157780,70.896282,0.032489
3,4,0.04,0.528739,1.578722,1.676403,0.545200,0.535484,0.578933,0.571257,0.015787,0.067056,57.872181,67.640257,0.041329
4,5,0.05,0.517226,1.546676,1.650457,0.534133,0.522804,0.569973,0.561567,0.015467,0.082523,54.667634,65.045732,0.049679
5,6,0.10,0.477334,1.452779,1.551618,0.501707,0.495535,0.535840,0.528551,0.072639,0.155162,45.277927,55.161830,0.084261
6,7,0.15,0.449971,1.353014,1.485417,0.467253,0.462955,0.512978,0.506686,0.067651,0.222813,35.301363,48.541674,0.111222
7,8,0.20,0.428245,1.277958,1.433552,0.441333,0.438761,0.495067,0.489705,0.063898,0.286710,27.795774,43.355199,0.132452
8,9,0.30,0.392886,1.192632,1.353245,0.411867,0.409843,0.467333,0.463084,0.119263,0.405974,19.263186,35.324528,0.161876
9,10,0.40,0.363135,1.084720,1.286114,0.374600,0.377766,0.444150,0.441755,0.108472,0.514446,8.471972,28.611389,0.174817


In [24]:
preds = aml.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [25]:
preds

predict,p0,p1
1,0.623613,0.376387
1,0.503573,0.496427
1,0.519108,0.480892
1,0.484665,0.515335
1,0.471497,0.528503
1,0.725794,0.274206
0,0.833806,0.166194
1,0.599826,0.400174
0,0.788753,0.211247
1,0.635705,0.364295
